In [1]:
config = {
    "user": "root",
    "password": "tseinfo",
    "database": "pri_database",
    "host": "localhost",
    "port": 3306
}

In [2]:
posts_query = "SELECT post_id, title, date, url, book_title FROM posts"

keywords_of_a_post_query = ("SELECT KEYWORD_NAME FROM posts_keywords pk "
                            "JOIN keywords k "
                            "ON pk.KEYWORD_ID = k.KEYWORD_ID "
                            "WHERE pk.POST_ID = {};")

posts_between_dates_query = posts_query + " WHERE date BETWEEN '{}' AND '{}'"

In [3]:
try:
    import mysql.connector
except ModuleNotFoundError:
    import sys
    print("Error: You need to `pip install mysql-connector-python`", file=sys.stderr)

In [4]:
cursor = None
cnx = None

def connect():
    global cursor, cnx
    #print("Connecting to database...")
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    cnx.autocommit = True

def disconnect():
    global cursor, cnx
    cursor.close()
    cnx.close()

def query(query, fetch=True, close_connexion=True, limit_count=None, limit_offset=0):
    global cursor, cnx
    if limit_count != None:
        query = query + " LIMIT {}, {}".format(limit_offset, limit_count)
    try:
        cursor.execute(query)
    except AttributeError: # cursor is None
        connect()
        cursor.execute(query)
    except mysql.connector.errors.ProgrammingError: # cursor is not connected
        connect()
        cursor.execute(query)
    finally:
        #print("executed query:\n", query) # useful for debugging
        if fetch:
            result = cursor.fetchall()
        if close_connexion:
            disconnect()
        if fetch:
            return result

In [5]:
def print_all_articles(limit=None):
    results = query(posts_query, limit_count=limit)
    for (post_id, title, date, url, book_title) in results:
        print("{:%d %b %Y}: {}".format(date, title))
        print(" | url:", url)

In [6]:
print_all_articles(limit=5)

26 Jan 2019: assessment of the key-reuse resilience of newhope
 | url: https://eprint.iacr.org/2019/075
26 Jan 2019: efficient and secure multiparty computation from fixed-key block ciphers
 | url: https://eprint.iacr.org/2019/074
26 Jan 2019: destructive privacy and mutual authentication in vaudenay's rfid model
 | url: https://eprint.iacr.org/2019/073
26 Jan 2019: zeroct: improving zerocoin with confidential transactions and more
 | url: https://eprint.iacr.org/2019/072
26 Jan 2019: repeatable oblivious shuffling of large outsourced data blocks
 | url: https://eprint.iacr.org/2019/071


In [7]:
def print_all_articles_with_info(limit=None):
    results = query(posts_query, limit_count=limit, close_connexion=False)
    for (post_id, title, date, url, book_title) in results:
        print("{:%d %b %Y}: {}".format(date, title))
        print(" | title:", title)
        print(" | book title:", book_title)
        keywords = []
        kw_results = query(keywords_of_a_post_query.format(post_id), close_connexion=False)
        for fields in kw_results:
            keywords.append(fields[0]) # fields[0] = keyword field
        print(" |", len(keywords), "keywords:", " ; ".join(keywords))
        disconnect()

In [8]:
print_all_articles_with_info(limit=5)

26 Jan 2019: assessment of the key-reuse resilience of newhope
 | title: assessment of the key-reuse resilience of newhope
 | book title: None
 | 4 keywords: public-key cryptography / post-quantum cryptography ; lattice based cryptography ; active attack ; side channels
26 Jan 2019: efficient and secure multiparty computation from fixed-key block ciphers
 | title: efficient and secure multiparty computation from fixed-key block ciphers
 | book title: None
 | 3 keywords: cryptographic protocols / random permutation mode ; secure computation ; 
26 Jan 2019: destructive privacy and mutual authentication in vaudenay's rfid model
 | title: destructive privacy and mutual authentication in vaudenay's rfid model
 | book title: None
 | 3 keywords: cryptographic protocols / rfid scheme ; security ; privacy
26 Jan 2019: zeroct: improving zerocoin with confidential transactions and more
 | title: zeroct: improving zerocoin with confidential transactions and more
 | book title: None
 | 4 keywords: 

In [9]:
def get_prints(limit=None):
    Ids, Titles, Book_titles, Keywords = [], [], [], []
    results = query(posts_query, limit_count=limit, close_connexion=False)
    for (post_id, title, date, url, book_title) in results:
        #print("{:%d %b %Y}: {}".format(date, title))
        #print(" | title:", title)
        #print(" | book title:", book_title)
        keywords = []
        kw_results = query(keywords_of_a_post_query.format(post_id), close_connexion=False)
        for fields in kw_results:
            keywords.append(fields[0]) # fields[0] = keyword field
        #print(" |", len(keywords), "keywords:", " ; ".join(keywords))
        Ids.append(post_id)
        Titles.append(title if title else "")                # if None, put "" instead
        Book_titles.append(book_title if book_title else "") # if None, put "" instead
        Keywords.append(keywords if keywords else "")        # if None, put "" instead
    disconnect()
    return Ids, Titles, Book_titles, Keywords

In [10]:
x_ids, x_titles, x_booktitles, x_keywords = get_prints()

In [11]:
print(x_titles[:5], x_booktitles[:5], x_keywords[:5], sep = '\n\n\n')

['assessment of the key-reuse resilience of newhope', 'efficient and secure multiparty computation from fixed-key block ciphers', "destructive privacy and mutual authentication in vaudenay's rfid model", 'zeroct: improving zerocoin with confidential transactions and more', 'repeatable oblivious shuffling of large outsourced data blocks']


['', '', '', '', '']


[['public-key cryptography / post-quantum cryptography', 'lattice based cryptography', 'active attack', 'side channels'], ['cryptographic protocols / random permutation mode', 'secure computation', ''], ['cryptographic protocols / rfid scheme', 'security', 'privacy'], ['applications / zero knowledge', 'rsa', 'discrete logarithm problem', 'anonymity'], ['cryptographic protocols / oblivious shuffling', 'data outsourcing', 'cloud computing']]


In [12]:
import datetime

def print_articles_publicated_between(start_year, end_year, limit=None):
    results = query(posts_between_dates_query.format(datetime.date(start_year, 1, 1),
                                                     datetime.date(end_year, 12, 31)),
                    limit_count=limit)
    for (post_id, title, date, url, book_title) in results:
        print("{:%d %b %Y}: {}".format(date, title))
        print(" | title:", title)
        print(" | book title:", book_title)

In [13]:
print_articles_publicated_between(2018, 2019, limit=10)

26 Jan 2019: assessment of the key-reuse resilience of newhope
 | title: assessment of the key-reuse resilience of newhope
 | book title: None
26 Jan 2019: efficient and secure multiparty computation from fixed-key block ciphers
 | title: efficient and secure multiparty computation from fixed-key block ciphers
 | book title: None
26 Jan 2019: destructive privacy and mutual authentication in vaudenay's rfid model
 | title: destructive privacy and mutual authentication in vaudenay's rfid model
 | book title: None
26 Jan 2019: zeroct: improving zerocoin with confidential transactions and more
 | title: zeroct: improving zerocoin with confidential transactions and more
 | book title: None
26 Jan 2019: repeatable oblivious shuffling of large outsourced data blocks
 | title: repeatable oblivious shuffling of large outsourced data blocks
 | book title: None
26 Jan 2019: uncle traps: harvesting rewards in a queue-based ethereum mining pool
 | title: uncle traps: harvesting rewards in a queue-b

In [14]:
cursor.close()
cnx.close()